# Plot up wind speed in NEMO forcing file

In [11]:
import xarray as xr
import matplotlib.pyplot as plt
import math
import numpy as np
import arrow
import cmocean

In [12]:
%matplotlib inline

## load data

In [13]:
ds = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaSurfaceAtmosphereFieldsV1')
ds

<xarray.Dataset>
Dimensions:    (gridX: 256, gridY: 266, time: 36120)
Coordinates:
  * time       (time) datetime64[ns] 2014-09-12 ... 2018-10-25T23:00:00
  * gridY      (gridY) float64 0.0 2.5e+03 5e+03 ... 6.575e+05 6.6e+05 6.625e+05
  * gridX      (gridX) float64 0.0 2.5e+03 5e+03 ... 6.35e+05 6.375e+05
Data variables:
    atmpres    (time, gridY, gridX) float32 ...
    precip     (time, gridY, gridX) float32 ...
    qair       (time, gridY, gridX) float32 ...
    solar      (time, gridY, gridX) float32 ...
    tair       (time, gridY, gridX) float32 ...
    therm_rad  (time, gridY, gridX) float32 ...
    u_wind     (time, gridY, gridX) float32 ...
    v_wind     (time, gridY, gridX) float32 ...
Attributes:
    acknowledgement:           MEOPAR, ONC, Compute Canada, Environment Canada
    cdm_data_type:             Grid
    Conventions:               CF-1.6, COARDS, ACDD-1.3
    coverage_content_type:     modelResult
    creator_email:             sallen@eos.ubc.ca
    creator_name:

## Calculate wind speed.  

In [29]:
ds.u_wind

<xarray.DataArray 'u_wind' (time: 36120, gridY: 266, gridX: 256)>
[2459627520 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2014-09-12 ... 2018-10-25T23:00:00
  * gridY    (gridY) float64 0.0 2.5e+03 5e+03 ... 6.575e+05 6.6e+05 6.625e+05
  * gridX    (gridX) float64 0.0 2.5e+03 5e+03 ... 6.325e+05 6.35e+05 6.375e+05
Attributes:
    _ChunkSizes:  [  1 266 256]
    level:        10 m above ground
    long_name:    U-Component of Wind
    short_name:   UGRD_10maboveground
    units:        m/s

In [25]:
U = np.array(math.sqrt(ds.u_wind**2 + ds.v_wind**2))
U

RuntimeError: NetCDF: DAP server error

In [24]:
A = np.array([[1, 1], [2, 2]])
A**2

array([[1, 1],
       [4, 4]])

In [ ]:
U